In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
import random

df=pd.read_csv('WIKI-AAPL.csv')
df=df.drop(['Date'],axis=1)
df.info()
run_cnt=1
result=pd.DataFrame()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9400 entries, 0 to 9399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         9400 non-null   float64
 1   High         9400 non-null   float64
 2   Low          9400 non-null   float64
 3   Close        9400 non-null   float64
 4   Volume       9400 non-null   float64
 5   Ex-Dividend  9400 non-null   float64
 6   Split Ratio  9400 non-null   float64
 7   Adj. Open    9400 non-null   float64
 8   Adj. High    9400 non-null   float64
 9   Adj. Low     9400 non-null   float64
 10  Adj. Close   9400 non-null   float64
 11  Adj. Volume  9400 non-null   float64
dtypes: float64(12)
memory usage: 881.4 KB


### Getting columns and drop rate percentages from User

In [19]:
col_names = []
while True:
    new_col = input("Column name: ");
    if new_col == 'STOP':
        break
    col_names.append(new_col)

Column name: Volume
Column name: Split Ratio
Column name: STOP


In [23]:
missing_prct = []
while True:
    missing_prct_one_round = []
    for col in col_names:
        missPct = input("Precentage for "+ col+": ")
        missing_prct_one_round.append(missPct)
    missing_prct.append(missing_prct_one_round)
    continueStr = input("Continue? Y/N: ")
    if continueStr == 'N' or continueStr == 'n':
        break

# missing_prct = [0.19,0.1,0.8,0.23]
# missing_prct = [0.21,0.04,0.12,0.11]
# missing_prct = [0.03,0.12,0.22,0.04]
missing_prct

Precentage for Volume: 0.7
Precentage for Split Ratio: 0.8
Continue? Y/N: n


[['0.7', '0.8']]

### Generating columns and drop rates randomly

In [15]:
col_sel_rate=round(random.uniform(0.1,0.99),2)
col_names = np.random.choice(df.columns, int(len(df.columns)*col_sel_rate), replace=False)

print('col_sel_rate: ',col_sel_rate)
print('No.of columns selected: ',int(len(df.columns)*col_sel_rate))
print('Columns list: ',col_names)
print(' ')

missing_prct = []
i=1
while True:
    missing_prct_one_round = []
    for col in col_names:
        missing_prct_one_round.append(round(random.uniform(0.1,0.99),2))
    
    print('Drop rates for round {}: '.format(i),missing_prct_one_round)
    
    missing_prct.append(missing_prct_one_round)
    continueStr = input("Continue? Y/N: ")
    if continueStr == 'N' or continueStr == 'n':
        break
    i=i+1
    
print(' ')
print('Array of drop rates:')
print(missing_prct)

col_sel_rate:  0.51
No.of columns selected:  6
Columns list:  ['Adj. Close' 'High' 'Ex-Dividend' 'Split Ratio' 'Adj. Volume' 'Adj. Open']
 
Drop rates for round 1:  [0.39, 0.33, 0.81, 0.72, 0.72, 0.98]
Continue? Y/N: y
Drop rates for round 2:  [0.7, 0.38, 0.21, 0.77, 0.54, 0.97]
Continue? Y/N: n
 
Array of drop rates:
[[0.39, 0.33, 0.81, 0.72, 0.72, 0.98], [0.7, 0.38, 0.21, 0.77, 0.54, 0.97]]


In [21]:
def defile_dataset(df, dictval):
    df_cp = df.copy()
    print(dictval)
    for mc,dr in dictval.items():
        s = len(df_cp[mc]) * float(dr)
        drop_ind = np.random.choice(np.arange(len(df_cp[mc])), size=int(s), replace=False)
        df_cp[mc].iloc[drop_ind] = np.nan
    return df_cp,list(dictval.keys())
def impute_once(df_orig,iter_val,input_md): 
    df_miss, cols = defile_dataset(df_orig,input_md)
    df_orig_slice = df_orig[cols]
    imputer = IterativeImputer(max_iter=iter_val)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    df_stg = df_miss.copy()
    df_stg_simp = df_miss.copy()
    imp_arr = imputer.fit_transform(df_stg)
    imp_simp = imp_mean.fit_transform(df_stg_simp)
    return df_orig_slice, df_miss[cols], pd.DataFrame(imp_arr[:,[df_orig.columns.get_loc(i) for i in cols]], columns=cols), pd.DataFrame(imp_simp[:,[df_orig.columns.get_loc(i) for i in cols]], columns=cols), imputer.n_iter_


In [24]:
data = []
for percentageBlock in missing_prct:
    i=0
    input_md ={} 
    for p in percentageBlock:
        input_md[col_names[i]] = p
        i=i+1
    
    iter_values = [100,200]
    for iter_val in iter_values:
        #print("--------------")
        #print("Iterations: ", iter_val)    
        df_og, df_def, df_imp, df_simimp, n_iter = impute_once(df,iter_val, input_md)
       
        for i in range(len(df_og.columns)):
            si_mse =  mean_squared_error(df_og[df_og.columns[i]], df_simimp[df_simimp.columns[i]])
            ii_mse =  mean_squared_error(df_og[df_og.columns[i]],df_imp[df_imp.columns[i]])
            col_name = df_og.columns[i]
            #print("MSE for ",df_og.columns[i], " by simple imputer is ",  si_mse, "and by iterative imputer is ", ii_mse)
         
            #print('Simple imputer')
            si_ttest_stat, si_ttest_pval = stats.ttest_ind(df_og[df_og.columns[i]],df_simimp[df_simimp.columns[i]])
            #print(df_og.columns[i],si_ttest_stat,si_ttest_pval)
        
            #print('Iterative imputer')
            ii_ttest_stat, ii_ttest_pval = stats.ttest_ind(df_og[df_og.columns[i]],df_imp[df_imp.columns[i]])
            #print(df_og.columns[i],ii_ttest_stat,ii_ttest_pval)
            data.append((run_cnt,
                         col_name,
                         input_md[col_name],
                         iter_val,
                         n_iter,
                         si_mse,
                         ii_mse,
                         si_ttest_pval,
                         ii_ttest_pval
                        ))
run_cnt= run_cnt + 1                                 
cols=['Run count','Column name', 'Missing percentage','Max Iterations','Iterations taken','si_mse','ii_mse','si_ttest_pval', 'ii_ttest_pval']
result_new = pd.DataFrame(data, columns=cols)
result=result.append(result_new,ignore_index=True)
result.to_csv('Output Results.csv',index=False)
result

{'Volume': '0.7', 'Split Ratio': '0.8'}
{'Volume': '0.7', 'Split Ratio': '0.8'}


,Run count,Column name,Missing percentage,Max Iterations,Iterations taken,si_mse,ii_mse,si_ttest_pval,ii_ttest_pval
0,1,Low,0.28,100,1,4.604189e+03,1.227118e+03,0.456044,0.676839
1,1,Close,0.6,100,1,1.089014e+04,1.048210e+03,0.807454,0.789053
2,1,Adj. Open,0.28,100,1,4.427203e+02,1.820601e-02,0.474337,0.997554
3,1,Ex-Dividend,0.92,100,1,7.009661e-03,7.006865e-03,0.397226,0.737750
4,1,High,0.63,100,1,1.148647e+04,8.945972e+02,0.561376,0.821977
...,...,...,...,...,...,...,...,...,...
63,4,Split Ratio,0.23,200,2,1.064288e-04,1.106135e-04,0.874167,0.896241
64,5,Volume,0.7,100,2,2.008163e+14,1.446089e+14,0.007260,0.031164
65,5,Split Ratio,0.8,100,2,3.252603e-04,9.984940e-04,0.015310,0.209845
66,5,Volume,0.7,200,2,1.876172e+14,1.349865e+14,0.297616,0.901538


In [18]:
df.corr()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Open,1.000000,0.999928,0.999900,0.999825,0.289030,0.087689,-0.002667,0.529323,0.529651,0.528404,0.528970,0.190706
High,0.999928,1.000000,0.999879,0.999920,0.290279,0.087312,-0.002607,0.528344,0.528743,0.527451,0.528073,0.193540
Low,0.999900,0.999879,1.000000,0.999913,0.286770,0.087664,-0.002670,0.530466,0.530821,0.529648,0.530208,0.186561
Close,0.999825,0.999920,0.999913,1.000000,0.288583,0.087353,-0.002526,0.529368,0.529779,0.528544,0.529189,0.190198
Volume,0.289030,0.290279,0.286770,0.288583,1.000000,0.012505,0.035989,0.608623,0.610538,0.605785,0.608021,0.538841
Ex-Dividend,0.087689,0.087312,0.087664,0.087353,0.012505,1.000000,-0.000612,0.052413,0.052231,0.052264,0.052160,-0.005333
Split Ratio,-0.002667,-0.002607,-0.002670,-0.002526,0.035989,-0.000612,1.000000,0.014283,0.014392,0.014252,0.014537,-0.003025
Adj. Open,0.529323,0.528344,0.530466,0.529368,0.608623,0.052413,0.014283,1.000000,0.999957,0.999944,0.999903,-0.056758
Adj. High,0.529651,0.528743,0.530821,0.529779,0.610538,0.052231,0.014392,0.999957,1.000000,0.999929,0.999954,-0.055433
Adj. Low,0.528404,0.527451,0.529648,0.528544,0.605785,0.052264,0.014252,0.999944,0.999929,1.000000,0.999955,-0.058911
